In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autoencoders/AutoEncoders/autoencoders.py
/kaggle/input/autoencoders/AutoEncoders/AutoEncoders.ipynb
/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/test_set.csv
/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/users.dat
/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/ratings.dat
/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/README
/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/.Rhistory
/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/training_set.csv
/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/movies.dat
/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/u.occupation
/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/u1.base
/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/u.info
/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/u4.test
/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/u.item
/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/README
/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [4]:
movies= pd.read_csv('/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/movies.dat',
                    sep='::',
                    header=None,
                    engine='python',
                   encoding='latin-1')
users= pd.read_csv('/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/users.dat',
                    sep='::',
                    header=None,
                    engine='python',
                   encoding='latin-1')

In [5]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
ratings=pd.read_csv('/kaggle/input/autoencoders/AutoEncoders/ml-1m/ml-1m/ratings.dat',
                    sep='::',
                    header=None,
                    engine='python',
                   encoding='latin-1')

In [8]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
#Preparing the train-test split
training_set= pd.read_csv('/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/u1.base',delimiter='\t')
test_set= pd.read_csv('/kaggle/input/autoencoders/AutoEncoders/ml-100k/ml-100k/u1.test',delimiter='\t')

In [10]:
training_set=np.array(training_set,dtype='int')
test_set=np.array(test_set,dtype='int')

In [11]:
training_set[0]

array([        1,         2,         3, 876893171])

In [12]:
#Getting the number of users and movies
nb_users=int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies= int(max(max(training_set[:,1]),max(test_set[:,1])))


In [13]:
#Converting the data into an array with users in lines and movies in columns:
def convert(data):
    new_data=[]
    for id_users in range(1,nb_users+1):
        id_movies=data[:,1][data[:,0]==id_users]
        id_ratings=data[:,2][data[:,0]==id_users]
        ratings=np.zeros(nb_movies)
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    return new_data
training_set=convert(training_set)
test_set=convert(test_set)

In [14]:
len(training_set),len(training_set[0])

(943, 1682)

In [15]:
len(test_set),len(test_set[0])

(943, 1682)

In [16]:
#Converting the data into Torch tensors
training_set= torch.FloatTensor(training_set)
test_set= torch.FloatTensor(test_set)

In [20]:
class SAE(nn.Module):
    def __init__(self,):
        super(SAE, self).__init__()
        self.fc1=nn.Linear(nb_movies, 20) #random number to represent features of movies.
        self.fc2=nn.Linear(20,10) # 20 neurons of previous layer, 10 neurons in the hidden layer.
        self.fc3=nn.Linear(10,20) #10 neurons as input from previous layer, 20 neurons as output to next layer,because decoding now.
        self.fc4= nn.Linear(20,nb_movies)
        self.activation= nn.Sigmoid() # as we use Sigmoid or Hyperbolic tangent function in an autoencoder
    def forward(self, x):
        x=self.activation(self.fc1(x)) #This returns the encoded input vector to the second hidden layer from the original input vector.
        x=self.activation(self.fc2(x))#This returns x into a shorter vector of 10  elements in the second hidden layer
        x=self.activation(self.fc3(x))# Now we are decoding from input vector of 10 elements to 20 elements as output.
        x=self.fc4(x) #Converts the previous layer 20 sized input vector into nb_movies sized output vector directly without any activation function applied.
        return x #Vector of our predicted ratings.

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr= 0.01, weight_decay=0.5)

So, we use this super function to be able to use the methods and classes from the nn module and the one we need and we are gonna use is the Linear class, which will make the different full connections between the layers

fc1  is the object of nn module that represents the full connection between this first input vector features and the first encoded vector.

In function forward:
We did first two encodings of our first 2 vectors features and then 2 decodings to get our reconstructed output vector in our output layer.

In [23]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7658)
epoch: 2loss: tensor(1.0966)
epoch: 3loss: tensor(1.0534)
epoch: 4loss: tensor(1.0384)
epoch: 5loss: tensor(1.0307)
epoch: 6loss: tensor(1.0266)
epoch: 7loss: tensor(1.0237)
epoch: 8loss: tensor(1.0217)
epoch: 9loss: tensor(1.0206)
epoch: 10loss: tensor(1.0196)
epoch: 11loss: tensor(1.0190)
epoch: 12loss: tensor(1.0182)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0177)
epoch: 15loss: tensor(1.0170)
epoch: 16loss: tensor(1.0168)
epoch: 17loss: tensor(1.0168)
epoch: 18loss: tensor(1.0164)
epoch: 19loss: tensor(1.0162)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0157)
epoch: 22loss: tensor(1.0160)
epoch: 23loss: tensor(1.0157)
epoch: 24loss: tensor(1.0158)
epoch: 25loss: tensor(1.0155)
epoch: 26loss: tensor(1.0156)
epoch: 27loss: tensor(1.0154)
epoch: 28loss: tensor(1.0151)
epoch: 29loss: tensor(1.0127)
epoch: 30loss: tensor(1.0113)
epoch: 31loss: tensor(1.0098)
epoch: 32loss: tensor(1.0071)
epoch: 33loss: tensor(1.0076)
epoch: 34loss: tens

 s = 0. .....Keeps track of users who have rated at least one movie.
 for id_user in range(nb_users): ....Takes indices from 0-942 in the training set
 if torch.sum(target.data>0)>0: ....Observations that dont contain zeros, or users who rated at least one movie
  target.require_grad= False .....Makes sure we dont compute the gradient with respect to the target.
   output[target==0]= 0 .....These values will not count in the computations of the error and hence wont have any impact in the error counting.
   s+=1. .....Increment s, as the user has rated at least one movie. since he/she came inside this loop.
 
 nb_movies/float(torch.sum(target.data>0)+ 1e-10) .....With 1e-10 we are making sure that the denominator is always >0


input= Variable(training_set[id_user]).unsqueeze(0) 
The 0 is the index of the new dimension. And the above statement will create a batch of a single input vector.
The batch can have several input vectors(batch learning).
We are going to update the weights after each observation going to the network. And therefore we are creating a batch
of one input vector. But we have to create this batch, otherwise it won't work.

Mean corrector represents the average of the error, but by only considering the movies that were rated, the movies that got atleast 1 to 5 ratings.

loss.data[0] : We access the data in the loss object and then we need to take the index of the data that contains the train loss, and the index of the train loss is 0. Hence data[0]

In [26]:
#Testing the SAE:
test_loss=0
s=0.
for id_user in range(nb_users):
    input= Variable(training_set[id_user]).unsqueeze(0) # We need the training set to predict ratings of movies the user has not watched yet.
    target= Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data>0)>0:
        output=  sae(input) #Stores predctions on the missing input data in the training set. Returns vector of predicted ratings.
        target.required_grad=False
        output[target==0]=0 #Only consider the ratings of the movies that are non zero ratings in the test set because target corresponds to the real ratings in the test set and we dont want to measure the future loss on these movies htat didn't get any ratings.
        loss=criterion(output,target)
        mean_corrector=nb_movies/float(torch.sum(target.data>0)+1e-10)
        test_loss+=np.sqrt(loss.data*mean_corrector)
        s+=1. #Only considering users that gave at least a non-zero rating.
print('test loss: '+str(test_loss/s))
    

test loss: tensor(0.9510)


Why do we use the training set?
We predict ratings for the movies that each user hasnot watched yet and then make predictions for the users and the movies and then compare those ratings to the ratings given for each movie and the user which were unwatched in the training set (present) but watched in the future(test set). We compare the predicted  ratings(training) and the actual ratings(test set).

output[target==0]=0 #Only consider the ratings of the movies that are non zero ratings in the test set because target corresponds to the real ratings in the test set and we dont want to measure the future loss on these movies that didn't get any ratings.

test loss: tensor(0.9510)
That means if we give a movie 3 stars, this system would predict we would give the movie somewhere between 2.05 to 3.95